# Determine degree of urbanisation for the LMS zones

Import modules

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from pylab import cm
from scipy.spatial.distance import pdist, squareform
import os

%matplotlib inline

In [ ]:
pd.set_option('display.max_columns', None)

Load the data and merge the data

In [ ]:
parent_dir = os.path.split(os.getcwd())[0] # Get parent directory

In [ ]:
lms_zones = gpd.read_file(parent_dir + '\\Data\\LMS\\Zone data\\LMS_Zones.shp') # load shapefile lms zones
zone_data = pd.read_excel(parent_dir + '\Data\LMS\Zone data\SEG data _LMS_versie_23_nov_2020.xlsx', 
                          sheet_name='ZON_18_LMS_versie_23_nov_2020') # load data of the lms zones
pc4 = gpd.read_file(parent_dir + '\Data\PC4 2022\cbs_pc4_2019_vol.gpkg') # load PC4 zones according to the CBS

In [ ]:
lms_gdf = lms_zones.merge(zone_data, on='ZONE_ID') # Merge lms zone data and shapefile
lms_gdf = lms_gdf.set_index(lms_gdf['ZONE_ID'])

In [ ]:
pc4.geometry.area.mean() / 10 ** 6

In [ ]:
# Keep only the zones in the Netherlands
lms_gdf = lms_gdf[lms_gdf.LAND == 1]
lms_gdf.plot()

Calculate the population density of each zone in inhabitants/ ha

In [ ]:
lms_gdf['Pop_dens'] = lms_gdf['INWONERS'] / lms_gdf['OPP']

In [ ]:
f, ax = plt.subplots(1, 1)
f.set_figwidth(13)
f.set_figheight(6)
ax.set_title('Population density LMS zones (capped at 85 p/ha)')
cmap = cm.get_cmap('YlOrRd')

lms_gdf.plot(ax=ax, column='Pop_dens', cmap=cmap, vmax=85,
         legend=True, legend_kwds={"label": "Population density [p/ha]"});

Determine the degree of urbanisation based on LMS documentation

In [ ]:
def deg_urban(gdf):
    """
    Determine the degree of urbanisation of the LMS zones
    """

    points = gdf[['XCOORD', 'YCOORD']] # df with all coordinated centroids zone
    
    # Distances between all points
    distances = pd.DataFrame(squareform(pdist(points)), columns=points.index, index=points.index) 

    # All neigbour zones within a distance of 3 km
    close_zones = distances[(distances <= 3000) & (distances != 0)]

    # Create new columns
    gdf['pop_dens_sur'] = np.zeros((len(gdf)))
    gdf['max_pop_dens'] = np.zeros((len(gdf)))
    gdf['deg_urba'] = np.zeros((len(gdf)))

    # Get population density surrounding zones
    for i in close_zones.index:
        zone_list = close_zones.loc[i]

        index_list = list(map(tuple, np.argwhere(~np.isnan(zone_list.values))))

        sur_area = gdf.loc[i]['OPP']
        sur_pop = gdf.loc[i]['INWONERS']
        
        for j in range(len(index_list)):
            zone_id = index_list[j][0] + 1

            sur_area += gdf.loc[zone_id]['OPP']
            sur_pop += gdf.loc[zone_id]['INWONERS']
        
        gdf.loc[i, 'pop_dens_sur'] = sur_pop / sur_area



    # Get zone with highest population density
    for i in close_zones.index:
        zone_list = close_zones.loc[i]
        index_list = list(map(tuple, np.argwhere(~np.isnan(zone_list.values))))

        pop_dens_dict = {}
        pop_dens_dict[i] = gdf.loc[i]['pop_dens_sur']
        
        for j in range(len(index_list)):
            zone_id = index_list[j][0] + 1
            pop_dens_dict[zone_id] = gdf.loc[zone_id]['pop_dens_sur']

        max_dens = max(pop_dens_dict.values())
        lms_gdf.loc[i, 'max_pop_dens'] = max_dens

    pop_range = [-1, 2.5, 6, 25, 50, 85, np.inf]

    for i in range(6):
        index_du = gdf.max_pop_dens[(gdf.max_pop_dens > pop_range[i]) & (gdf.max_pop_dens <= pop_range[i + 1])].index
        gdf.loc[index_du, 'deg_urba'] = i + 1
    
    return gdf
        


In [ ]:
lms_gdf = deg_urban(lms_gdf)

Plot the degree of urbanisation on the map

In [ ]:
f, ax = plt.subplots(1, 1)
f.set_figwidth(13)
f.set_figheight(6)

cmap = cm.get_cmap('YlOrRd', 6)

lms_gdf.plot(ax=ax, column='deg_urba', cmap=cmap, vmax=6.5,
         legend=True, legend_kwds={"label": "Degree of urbanisation"})

ax.set_facecolor('lightskyblue')
ax.set_title('Degree of urbanisation - LMS')

ax.set_yticks([])
ax.set_xticks([])


And now compare the degree of urbanisation of LMS with the CBS

In [ ]:
index_gdf = pc4.stedelijkheid[pc4.stedelijkheid < 1].index
pc4.loc[index_gdf, 'stedelijkheid'] = np.nan

In [ ]:
f, ax = plt.subplots(1, 2)
f.set_figwidth(13)
f.set_figheight(6)

cmap = cm.get_cmap('YlOrRd', 6)
cmap_cbs = cm.get_cmap('YlOrRd_r', 5)

lms_gdf.plot(ax=ax[0], column='deg_urba', cmap=cmap, vmax=6.5,
         legend=True, legend_kwds={"label": "Degree of urbanisation LMS", "ticks": np.arange(1, 7)})

pc4.plot(ax=ax[1], column='stedelijkheid', legend=True, cmap=cmap_cbs,
         legend_kwds={"label": "Degree of urbanisation CBS", "ticks": np.arange(1, 6)})

ax[0].set_title('Degree of urbanisation according to the LMS')
ax[1].set_title('Degree of urbanisation according to the CBS')

ax[0].set_facecolor('lightskyblue')
ax[1].set_facecolor('lightskyblue')

ax[0].set_yticks([])
ax[0].set_xticks([])

ax[1].set_yticks([])
ax[1].set_xticks([])


All zones have been assigned a degree of urbanisation

In [ ]:
lms_gdf[lms_gdf.deg_urba == 0]

In [ ]:
lms_gdf

In [ ]:
lms_gdf = lms_gdf.drop(columns='ZONE_ID')

Save dataframe with degree of urbanisation

In [ ]:
# lms_gdf.to_file(parent_dir + '\\Data\\New\\lms_zone_du_new2.shp')